In [1]:
import pandas as pd
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Bishan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bishan
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv("../data/raw/reddit_emotions.csv")
df

,title,selftext,subreddit,created_utc,id,emotional_score
0,He’s gone… but I walked the stage in his old w...,"Last week, I graduated college, the first in m...",gratitude,2025-07-17 18:31:20,1m26xq1,1.0
1,I'm grateful that ChatGPT presents information...,There's a venue I want to visit today. All I ...,gratitude,2025-07-17 19:24:54,1m286ia,1.0
2,Grateful for a cup of nice orange juice.,I don't drink alcohol. So when I'm feeling blu...,gratitude,2025-07-17 20:05:28,1m2971s,1.0
3,I am grateful for a deep breath that instantly...,Gratitude Practice Day 61,gratitude,2025-07-17 15:30:42,1m23j0l,1.0
4,How to be grateful when your feeling behind in...,Hey everyone! I’ve been trying to be grateful ...,gratitude,2025-07-17 09:46:46,1m1xz7r,1.0
...,...,...,...,...,...,...
6128,Is this wrong of me?,This is going to sound wrong but after seeing ...,SuicideWatch,2025-07-14 22:33:05,1lzrzj5,0.0
6129,Tomorrow,"Tomorrow, I will drop my kids off, go to work,...",SuicideWatch,2025-07-14 08:34:56,1lzbkr8,0.0
6130,Everything is just terrible..,"My health is not doing great, debt sky high, n...",SuicideWatch,2025-07-14 14:54:53,1lzhuo3,0.0
6131,I want to die because of my intrusive thoughts,It's hard for me to even write this out becaus...,SuicideWatch,2025-07-14 12:19:42,1lzfi1o,0.0


In [3]:
df.isna().sum()

title              0
selftext           0
subreddit          0
created_utc        0
id                 0
emotional_score    0
dtype: int64

In [4]:
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if ((word not in STOPWORDS) and (word != "grateful"))]
    return " ".join(tokens)

In [5]:
df.iloc[0]['selftext']

'Last week, I graduated college, the first in my entire family to do it.\n\nBut my dad wasn’t there to see it. He passed away during my freshman year. No warning. No goodbye.\n\nMy father was a working-class man through and through. He wore the same pair of steel-toe boots to work for over a decade, pulled double shifts, skipped birthdays, and never once complained. All so that *I* could have a different kind of future.\n\n“Get that degree, son,” he used to tell me. “Make it count.”\n\nThe night before graduation, I found those old boots buried in my closet. Scuffed. Worn through. Still faintly smelling like oil and garage dust. I held them for a while, and cried. Then I made a choice.\n\nI wore them under my cap and gown.\n\nSize 11s on my size 9 feet. They didn’t match anything. They didn’t need to.\n\nWalking across that stage, I felt like every step carried not just my effort, but his. His sacrifices. His love. His belief in me, even when I didn’t see it in myself.\n\nWhen I hugged

In [6]:
df["full_text"] = df["title"].fillna("") + " " + df["selftext"].fillna("")
df["clean_text"] = df["full_text"].apply(clean_text)

In [7]:
df_clean = df[["clean_text", "subreddit", "emotional_score"]]
df_clean

,clean_text,subreddit,emotional_score
0,hes gone walked stage old work boots im last w...,gratitude,1.0
1,im chatgpt presents information peoples websit...,gratitude,1.0
2,cup nice orange juice dont drink alcohol im fe...,gratitude,1.0
3,deep breath instantly calms gratitude practice...,gratitude,1.0
4,feeling behind life hey everyone ive trying al...,gratitude,1.0
...,...,...,...
6128,wrong going sound wrong seeing suicide videos ...,SuicideWatch,0.0
6129,tomorrow tomorrow drop kids go work get come h...,SuicideWatch,0.0
6130,everything terrible health great debt sky high...,SuicideWatch,0.0
6131,want die intrusive thoughts hard even write im...,SuicideWatch,0.0


In [8]:
df["subreddit"].value_counts()

subreddit
BenignExistence    969
TodayIAmHappy      806
lonely             753
gratitude          674
SuicideWatch       649
Emotions           516
happy              480
Anxiety            459
vent               428
offmychest         399
Name: count, dtype: int64

In [9]:
df_clean.to_csv("../data/processed/reddit_emotions_cleaned.csv", index=False)